In [69]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [70]:
stories_json = []
with open('./output_storiesv2.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

407
dict_keys(['index', 'title', 'broadcasting_date', 'abstracts_list'])


In [71]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...


In [72]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 瀕死を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [73]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
3      [のび太, ドラえもん, 部屋, 昼寝, 音, となり, 家, 改築, 工事, の, しずか...
4      [そうじ, せんたく, 町内会, 会合, 毎日, 大忙し, ママ, 体, ひとつ, ため息,...
                             ...                        
402    [しずか, 遊び, ジャイアン, 意味, ボコボコ, のび太, 家, 部屋, 中, ドラえも...
403    [スネ夫, 手, 恐竜, きょう, りゅう, プラモデル, じまん, のび太, プラモデル,...
404    [秋, ため, 別荘, べっそう, スネ夫, 季節感, 顔, バカ, のび太, 今年, ふり...
405    [空き地, サッカー, ジャイアン, スネ夫, たち, ジャイアン, ボール, 神成, さん...
406    [使い, の, ちゅ, 腕木, げん, そう, ジャイアン, 別, 道, のび太, 空き, ...
Name: abstract_joined, Length: 407, dtype: object


In [74]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [75]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ -8.972734    -9.708607  ]
 [  5.300853     8.616312  ]
 [-15.768965   -20.648449  ]
 [ -6.898759    -9.275977  ]
 [  2.35441      6.562129  ]
 [ -4.9992476  -10.3394    ]
 [-19.496748   -26.160566  ]
 [-18.312569   -24.956028  ]
 [-17.597115   -25.267895  ]
 [-10.557319   -18.38098   ]
 [-12.1486635   -9.144155  ]
 [  8.393354     9.850264  ]
 [ 20.384703    25.028059  ]
 [  4.173272     4.4632006 ]
 [  3.8401752    6.4105997 ]
 [  4.576984     5.362897  ]
 [ -2.2739224   -0.8994301 ]
 [ -4.5259314   -2.8484843 ]
 [ 10.951606    17.946033  ]
 [-10.369805    -9.519137  ]
 [-10.064304   -19.205935  ]
 [-10.17395    -17.614893  ]
 [-21.333292   -27.393562  ]
 [ -7.293543   -12.912419  ]
 [-19.127642   -23.61627   ]
 [  6.375417    11.610197  ]
 [ -5.2962193   -7.101452  ]
 [ -1.0313346   -5.561567  ]
 [-22.424967   -28.135279  ]
 [  4.379441    10.065784  ]
 [ 14.081441    15.048232  ]
 [  1.4044155    4.68232   ]
 [ -5.834649   -10.784898  ]
 [-19.058344   -25.312418  ]
 [  6.587536  

In [76]:
vector_df = pd.DataFrame.from_dict({
    'vector_x': [v[0] for  v in vector_enbedded],
    'vector_y': [v[1] for  v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [77]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [80]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()